# BB84 on Qiskit — Aer (local) + IBM Runtime (online)

This notebook implements the BB84 protocol end‑to‑end using:
- **Aer (local)** — fully offline via `AerSimulator`.
- **IBM Runtime (online)** — optional Sampler executor.

Toggle with `executor="aer"` *(default)* or `executor="runtime"` in `BB84(...)`.

In [ ]:
import numpy as np
from typing import List, Tuple, Optional
from dataclasses import dataclass

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

print("Imports OK")

In [ ]:
# --- Random helpers ---
def create_random_generator(seed: Optional[int] = None) -> np.random.Generator:
    return np.random.default_rng(seed)

def create_random_bits(rng: np.random.Generator, m: int) -> np.ndarray:
    return rng.integers(0, 2, size=m, dtype=np.int8)

def create_random_bases(rng: np.random.Generator, m: int) -> np.ndarray:
    # 0 -> Z, 1 -> X
    return rng.integers(0, 2, size=m, dtype=np.int8)

# --- 1‑qubit BB84 circuit ---
def one_qubit_bb84_circuit(bit: int, alice_basis: int, bob_basis: int) -> QuantumCircuit:
    """Prepare Alice's qubit (Z/X) and measure in Bob's basis (Z/X)."""
    qc = QuantumCircuit(1, 1)
    # Alice prep
    if alice_basis == 0:       # Z basis
        if bit == 1:
            qc.x(0)            # |1>
    else:                      # X basis
        qc.h(0)                # |+>
        if bit == 1:
            qc.z(0)            # |->
    # Bob measurement
    if bob_basis == 1:         # measure in X
        qc.h(0)
    qc.measure(0, 0)
    return qc

In [ ]:
# --- Local Aer executor (shots=1 per circuit) ---
def run_single_shot_batch(circuits: List[QuantumCircuit]) -> List[int]:
    sim = AerSimulator()
    tcs = [transpile(c, sim) for c in circuits]
    res = sim.run(tcs, shots=1).result()
    bits = []
    for i in range(len(tcs)):
        counts = res.get_counts(i)        # {'0':1} or {'1':1}
        bits.append(int(next(iter(counts))))
    return bits

In [ ]:
# --- IBM Runtime (online) executor ---
# Requires: qiskit-ibm-runtime installed and account saved with QiskitRuntimeService.save_account(...)

def pick_ibm_backend(service, backend_name=None):
    if backend_name:
        return service.backend(backend_name)
    sims = service.backends(simulator=True, operational=True)
    if sims:
        try:
            sims = sorted(sims, key=lambda b: getattr(b.status(), "pending_jobs", 0))
        except Exception:
            pass
        return sims[0]
    qpus = service.backends(simulator=False, operational=True)
    if not qpus:
        raise RuntimeError("No operational IBM backends found for this account/instance.")
    try:
        qpus = sorted(qpus, key=lambda b: getattr(b.status(), "pending_jobs", 0))
    except Exception:
        pass
    return qpus[0]

def run_single_shot_batch_runtime(circuits: List[QuantumCircuit], *, runtime_service=None, backend_name=None, shots=1):
    try:
        from qiskit_ibm_runtime import SamplerV2 as Sampler, Session, QiskitRuntimeService
    except Exception:
        from qiskit_ibm_runtime import Sampler, Session, QiskitRuntimeService
    service = runtime_service or QiskitRuntimeService()
    backend = pick_ibm_backend(service, backend_name=backend_name)
    with Session(service=service, backend=backend) as session:
        sampler = Sampler(mode=session)
        job = sampler.run(circuits, shots=shots)
        result = job.result()
    bits = []
    for i in range(len(circuits)):
        pub = result[i]
        try:
            counts = pub.data.meas.get_counts()
        except Exception:
            counts = pub.join_data().get_counts()
        bits.append(int(next(iter(counts))[-1]))
    return bits

In [ ]:
@dataclass
class BB84Result:
    key_bits: List[int]
    qber_sample: float
    raw_transmissions: int
    sifted_size_before_sample: int
    sample_indices: List[int]
    kept_indices: List[int]

def sift(alice_bits: np.ndarray, alice_bases: np.ndarray,
         bob_bases: np.ndarray, bob_bits: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    match = (alice_bases == bob_bases)
    index = np.nonzero(match)[0]
    return alice_bits[index], bob_bits[index], index

def sample_and_verify(a_sift: np.ndarray, b_sift: np.ndarray, s: int, rng: np.random.Generator,
                      qber_threshold: float = 0.02):
    if len(a_sift) < s:
        raise ValueError(f"Not enough sifted bits to sample: have {len(a_sift)}, need s={s}")
    sample_idx = rng.choice(len(a_sift), size=s, replace=False)
    mism = int(np.sum(a_sift[sample_idx] != b_sift[sample_idx]))
    qber = mism / s
    mask = np.ones(len(a_sift), dtype=bool)
    mask[sample_idx] = False
    kept_b = b_sift[mask]
    if qber > qber_threshold:
        raise ValueError(f"QBER too high in sample: {qber:.3f} > {qber_threshold:.3f}")
    kept_indices = np.nonzero(mask)[0].tolist()
    return kept_b.astype(int).tolist(), qber, sample_idx.tolist(), kept_indices

In [ ]:
# --- BB84 with toggle: executor='aer' (default) or 'runtime' ---
def BB84(n: int, s: int, *, seed: Optional[int] = None, batch_size: int = 1024,
         executor: str = "aer", runtime_service=None, backend_name: Optional[str] = None,
         qber_threshold: float = 0.02) -> BB84Result:
    if n <= 0 or s < 0:
        raise ValueError("n must be >0 and s >=0")
    rng = create_random_generator(seed)
    a_sift_all, b_sift_all = [], []
    raw_total = 0
    while True:
        m = batch_size
        alice_bits  = create_random_bits(rng, m)
        alice_bases = create_random_bases(rng, m)
        bob_bases = create_random_bases(rng, m)
        circs = [one_qubit_bb84_circuit(int(alice_bits[i]), int(alice_bases[i]), int(bob_bases[i])) for i in range(m)]
        if executor.lower() == "aer":
            bob_bits = np.array(run_single_shot_batch(circs), dtype=np.int8)
        elif executor.lower() == "runtime":
            bob_bits = np.array(run_single_shot_batch_runtime(
                circs, runtime_service=runtime_service, backend_name=backend_name, shots=1
            ), dtype=np.int8)
        else:
            raise ValueError("executor must be 'aer' or 'runtime'")
        raw_total += m
        alice_sift, bob_sift, _ = sift(alice_bits, alice_bases, bob_bases, bob_bits)
        if len(alice_sift):
            a_sift_all.append(alice_sift); b_sift_all.append(bob_sift)
        if sum(len(x) for x in a_sift_all) >= n + s:
            break
    alice_sift_cat = np.concatenate(a_sift_all) if a_sift_all else np.array([], dtype=np.int8)
    bob_sift_cat = np.concatenate(b_sift_all) if b_sift_all else np.array([], dtype=np.int8)
    kept_bits, qber, sample_idx, kept_idx = sample_and_verify(alice_sift_cat, bob_sift_cat, s, rng, qber_threshold=qber_threshold)
    if len(kept_bits) < n:
        raise RuntimeError(f"After sampling, not enough bits remain: have {len(kept_bits)}, need {n}")
    key_bits = kept_bits[:n]
    return BB84Result(
        key_bits=key_bits,
        qber_sample=qber,
        raw_transmissions=raw_total,
        sifted_size_before_sample=len(alice_sift_cat),
        sample_indices=sample_idx,
        kept_indices=kept_idx[:n]
    )

### Usage

**Local (offline, default):**
```python
res = BB84(n=100, s=10, seed=42, executor="aer")
```

**IBM Runtime (online):**
```python
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()  # after save_account(...)

res = BB84(
    n=100, s=10, seed=123,
    executor="runtime",
    runtime_service=service,
    backend_name=None  # or a specific backend name
)
print(res.qber_sample, len(res.key_bits))
```

In [ ]:
# --- Demo: local Aer ---
res = BB84(n=100, s=10, seed=42, executor="aer", batch_size=1024)
print("Raw transmissions:", res.raw_transmissions)
print("Sifted (before sampling):", res.sifted_size_before_sample)
print("Sample QBER:", res.qber_sample)
print("First 32 key bits:", ''.join(map(str, res.key_bits[:32])), "...")
print("Key length:", len(res.key_bits))